In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        if filename.find('.zip')>-1:
            zipfile.ZipFile(os.path.join(dirname, filename)).extractall('/kaggle/working/'+filename.replace('.zip',''))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import Non-PyTorch Modules will be used in the lab

import time
from imageio import imread
from matplotlib.pyplot import imshow
import matplotlib.pylab as plt
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import random
import numpy as np

In [ ]:
# Import PyTorch Modules will be used in the lab

import torch 
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision import transforms
import torch.nn as nn
torch.manual_seed(0)

In [ ]:
# Keras Modules

import keras
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.layers.convolutional import Conv2D # to add convolutional layers
from keras.layers.convolutional import MaxPooling2D # to add pooling layers
from keras.layers import Flatten # to flatten data for fully connected layers

In [ ]:
# !unzip '../input/aerial-cactus-identification/test.zip'
# !unzip '../input/aerial-cactus-identification/test.zip'

In [ ]:
work_dir = '/kaggle/working'
train_data_dir = '/kaggle/working/train/train'
test_data_dir = '/kaggle/working/test/test'
train_csv = '/kaggle/input/aerial-cactus-identification/train.csv'
sample_csv_path  = '/kaggle/input/aerial-cactus-identification/sample_submission.csv'

In [ ]:
class Dataset():
    def __init__(self,data_dir,data_csv,transform=None):
        self.csv=pd.read_csv(data_csv)
        self.data_dir=data_dir
        self.transform=transform
        self.len=self.csv.shape[0]
        
    def __len__(self):
        return self.len
    
    def __getitem__(self,idx):
        img_name=self.data_dir+self.csv.iloc[idx,0]
        image=Image.open(img_name)
        y=self.csv.iloc[idx,1]
        if self.transform:
            image=self.transform(image)
        return image,y

In [ ]:
#train_csv='../input/aerial-cactus-identification/train.csv'
train = pd.read_csv(train_csv)
sample = pd.read_csv(sample_csv_path)
sample['has_cactus'] = sample['has_cactus'].astype(str)
train['has_cactus'] = train['has_cactus'].astype(str)

In [ ]:
train_data = Dataset(train_data_dir,train_csv)

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)
batch_size = 200

In [ ]:
train_generator = datagen.flow_from_dataframe(dataframe = train[:15001],
                                            directory = train_data_dir,
                                            x_col = 'id',
                                            y_col = 'has_cactus',
                                            class_mode = 'binary',
                                            batch_size = batch_size,
                                            target_size = (32,32))

validation_generator = datagen.flow_from_dataframe(dataframe = train[15000:],
                                                 directory = train_data_dir,
                                                 x_col = 'id',
                                                y_col = 'has_cactus',
                                                 class_mode = 'binary',
                                                 batch_size = 50,
                                                target_size = (32,32))


In [ ]:
def model(num_classes):
    models = Sequential()
    models.add(Conv2D(16, (3, 3), strides = (1, 1), activation = 'relu', input_shape = (32, 32, 3)))
    models.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    models.add(Conv2D(64, (3, 3), strides = (1, 1), activation = 'relu', input_shape = (32, 32, 3)))  
    models.add(MaxPooling2D(pool_size = (2, 2), strides = (1, 1)))
    models.add(Conv2D(128, (3, 3), strides = (1, 1), activation = 'relu', input_shape = (32, 32, 3)))  
    models.add(MaxPooling2D(pool_size = (2, 2), strides = (1, 1)))
    models.add(Conv2D(256, (3, 3), strides = (1, 1), activation = 'relu', input_shape = (32, 32, 3)))  
    models.add(MaxPooling2D(pool_size = (2, 2), strides = (1, 1)))
  
    models.add(Flatten())
    models.add(Dense(512, activation = 'relu'))
    models.add(Dense(num_classes, activation = 'sigmoid'))
    models.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return models

In [ ]:
model = model(1)

In [ ]:
model.summary()

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor = 'loss', 
                                        patience = 5, 
                                        restore_best_weights = True,
                                        verbose = 2)
model.fit_generator(generator = train_generator,
                    epochs = 50, 
                    verbose = 2, 
                    validation_data = validation_generator,
                   callbacks = [callback]
                   )

In [ ]:
train_history = model.history.history

In [ ]:
plt.plot(train_history['accuracy'])
plt.show()
plt.plot(train_history['val_accuracy'])

In [ ]:
from tqdm import tqdm_notebook
import tensorflow as tf
preds = []
for fname in tqdm_notebook(sample['id']):
    path = os.path.join(test_data_dir,fname)
    
    img_pil = Image.open(path)
    image = np.array(img_pil)/255
    pred = model.predict(image[tf.newaxis, ...])
    preds.append(pred)

In [ ]:
submission = sample
preds = np.array(preds).reshape(4000, -1)

In [ ]:
submission['has_cactus'] = preds
submission['has_cactus'] = submission['has_cactus'].astype(int)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index = False)